In [1]:
import os
import numpy as np
import pandas as pd
from model import MRIA_IO as MRIA
from table import io_basic
from tqdm import tqdm
data_path = os.path.join('..','Data')

In [2]:
input_file = os.path.join(data_path,'isolated_od_scenarios','single_mode','single_edge_failures_od_losses_rail_min_100_percent_disrupt.csv')

In [3]:
output_path = os.path.join('..','results')

"""Specify disruption"""
output_dir = os.path.join(
    output_path,
    'agriculture_failure')

### Get regional mapper dictionary, to make everything compatible

In [4]:
reg_mapper = pd.read_excel(os.path.join(data_path,'economic_IO_tables','input','sh_cou_06_16.xls'),
                          sheet_name='reg_mapper',header=None)
reg_mapper = dict(zip(reg_mapper[0], reg_mapper[1]))

### Create input-output database file to be used with optimization model

In [5]:
%%time
""" Specify file path """
filepath = os.path.join(data_path, 'economic_IO_tables','output', 'IO_ARGENTINA.xlsx')

regions = ['Ciudad de Buenos Aires', 'Buenos Aires', 'Catamarca', 'Cordoba',
       'Corrientes', 'Chaco', 'Chubut', 'Entre Rios', 'Formosa', 'Jujuy',
       'La Pampa', 'La Rioja', 'Mendoza', 'Misiones', 'Neuquen', 'Rio Negro',
       'Salta', 'San Juan', 'San Luis', 'Santa Cruz', 'Santa Fe',
       'Santiago del Estero', 'Tucuman', 'Tierra del Fuego']

regions = [x.replace(' ','_') for x in regions]

"""Create data input"""
DATA = io_basic('Argentina', filepath,regions)
DATA.prep_data()

Wall time: 11.8 s


### RUN MRIA model for all baseline

In [6]:
%%time
"""Run model and create some output"""
output = pd.DataFrame()

disr_dict_fd = {}
disr_dict_sup = {}

"""Create model"""
MRIA_RUN = MRIA(DATA.name, DATA.regions, DATA.sectors, list_fd_cats=['FinDem'])

"""Define sets and alias"""
# CREATE SETS
MRIA_RUN.create_sets()

# CREATE ALIAS
MRIA_RUN.create_alias()

""" Define tables and parameters"""
MRIA_RUN.baseline_data(DATA, disr_dict_sup, disr_dict_fd)
MRIA_RUN.impact_data(DATA, disr_dict_sup, disr_dict_fd)

status = MRIA_RUN.run_impactmodel(output='print')
print(status)

#MRIA_RUN.run_basemodel()

"""Get base line values"""
output['x_in'] = pd.Series(MRIA_RUN.X.get_values())
output.index.names = ['region', 'sector']

Ipopt 3.11.1: max_iter=7500
warm_start_init_point=yes
warm_start_bound_push=1e-06
warm_start_mult_bound_push=1e-06
mu_init=1e-06


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

NOTE: You are using Ipopt by default with the MUMPS linear solver.
      Other linear solvers might be more efficient (see Ipopt documentation).


This is Ipopt version 3.11.1, running with linear solver mumps.

Number of nonzeros in equality constraint Jacobian...:   166656
Number of nonzeros in inequality constraint Jacobian.:     1152
Number of nonzeros in Lagrangian Hessian.............:      768

Total number of variables............................:     1536
             

In [7]:
%%time
"""Run model and create some output"""
disr_dict_fd = {}
collect_outputs = {}

sum_disr = 0

for event in tqdm(list(reg_mapper.values())):
    
    """Run model and create some output"""
    disr_dict_fd = {}
    disr_dict_sup = {(event, 'A'): 0.5}

    """Create model"""
    MRIA_RUN = MRIA(DATA.name, DATA.regions, DATA.sectors, list_fd_cats=['FinDem'])

    """Define sets and alias"""
    # CREATE SETS
    MRIA_RUN.create_sets()

    # CREATE ALIAS
    MRIA_RUN.create_alias()

    """ Define tables and parameters"""
    MRIA_RUN.baseline_data(DATA, disr_dict_sup, disr_dict_fd)
    MRIA_RUN.impact_data(DATA, disr_dict_sup, disr_dict_fd)

    """Get direct losses """
    disrupt = pd.DataFrame.from_dict(disr_dict_sup, orient='index')
    disrupt.reset_index(inplace=True)
    disrupt[['region', 'sector']] = disrupt['index'].apply(pd.Series)
    disrupt.drop('index', axis=1, inplace=True)
    disrupt = 1 - disrupt.groupby(['region', 'sector']).sum()
    disrupt.columns = ['shock']

    output['dir_losses'] = (disrupt['shock']*output['x_in']).fillna(0)*-1

    status = MRIA_RUN.run_impactmodel()
    if status.key != 'ok':
        continue
        
    output['x_out'] = pd.Series(MRIA_RUN.X.get_values())
    output['total_losses'] = (output['x_out'] - output['x_in'])
    output['ind_losses'] = (output['total_losses'] - output['dir_losses'])

    output.to_csv(os.path.join(output_dir, '{}.csv'.format(event)))
    
    prov_impact = output.groupby(level=0, axis=0).sum()[['dir_losses','total_losses','ind_losses']]
    collect_outputs[event] = prov_impact

    total_losses_sum = (output['total_losses'].sum().sum())
    print('{} results in {} Million USD losses'.format(event,total_losses_sum))

  0%|                                                                                           | 0/24 [00:00<?, ?it/s]

Buenos_Aires results in -5584.7973496812 Million USD losses


  4%|███▍                                                                               | 1/24 [00:14<05:38, 14.73s/it]

Catamarca results in -5.341874394180113 Million USD losses


  8%|██████▉                                                                            | 2/24 [00:27<05:09, 14.08s/it]

...

Tierra_del_Fuego results in -6.530053971146986 Million USD losses


 96%|██████████████████████████████████████████████████████████████████████████████▌   | 23/24 [04:48<00:12, 12.59s/it]

Tucuman results in -13.787764854957677 Million USD losses


100%|██████████████████████████████████████████████████████████████████████████████████| 24/24 [05:01<00:00, 12.50s/it]


Wall time: 5min 1s


In [11]:
pd.concat(collect_outputs).to_csv('D:\\Dropbox\\OIA\\Argentina\\results\\agriculture_failure\\all_regions.csv')